In [ ]:
import os
import re
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from ncps.torch import LTC
from ncps.wirings import AutoNCP
import random
import matplotlib.pyplot as plt
import warnings
from sklearn.metrics import root_mean_squared_error, mean_absolute_percentage_error
warnings.filterwarnings('ignore')
import sys
import json

In [ ]:
def custom_sort_key_dirs(filename):
    match = re.search(r"(\d+)_(\d+)", filename)
    if match:
        return (int(match.group(1)), int(match.group(2)))
    return (float('inf'), float('inf'))

def custom_sort_key_files(filename):
    match = re.search(r"(\d+)", filename)
    if match:
        return int(match.group(1))
    return float('inf')

def load_data_from_txt_folder(folder_path):
    data = []
    files = sorted(os.listdir(folder_path), key=custom_sort_key_files)
    for file in files:
        if file.endswith('.txt'):
            df = pd.read_csv(os.path.join(folder_path, file), header=None, delim_whitespace=True)
            bitrate = df.iloc[:, 0].values.reshape(-1, 1)
            data.append(bitrate)
    return data

class BitrateTimeSeriesDataset(Dataset):
    def __init__(self, data, input_steps=2, neigh_dict=None):
        self.input_steps = input_steps
        self.data = data
        self.neigh_dict = neigh_dict
        self.samples = []
        connection_ids = np.arange(82).reshape(-1, 1)

        for i in range(input_steps, len(data)):
            x = []
            for j in range(input_steps, 0, -1):
                bitrate = data[i - j]
                x_t = np.concatenate([bitrate, connection_ids], axis=1)  
                x.append(x_t)
            x = np.stack(x, axis=0) 
            y = data[i].flatten()  
            self.samples.append((x, y))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        x, y = self.samples[idx]
        x = np.stack([add_neighbour_channels(frame, self.neigh_dict)
                  for frame in x], axis=0) 
        x_t = torch.tensor(x, dtype=torch.float32)  
        y_t = torch.tensor(y, dtype=torch.float32) 
        return x_t, y_t


class TrafficPredictionNetwork(nn.Module):
    def __init__(self, embedding_dim, ltc_units, dense_units):
        super().__init__()

        self.embedding = nn.Embedding(82 + 1, embedding_dim, padding_idx=82) 
        self.ltc_input_size = 1 + embedding_dim
        self.ltc_input_size = 1 + (1 + 7) * embedding_dim 


        self.ltc = LTC(input_size=self.ltc_input_size, units=ltc_units, return_sequences=True)

        self.dense1 = nn.Linear(ltc_units, dense_units[0])
        self.dense2 = nn.Linear(dense_units[0], dense_units[1])
        self.dense3 = nn.Linear(dense_units[1], 1)
        self.tanh = nn.Tanh()
        

    
    def forward(self, x, hx=None):
        bitrate = x[..., 0].unsqueeze(-1)             
        id_inputs = x[..., 1:].long()                 
        embedded = self.embedding(id_inputs)          
        embedded = embedded.view(*embedded.shape[:3], -1)  

        inputs = torch.cat([bitrate, embedded], dim=-1)  
        B, S, L, F = inputs.shape
        inputs = inputs.view(B, S * L, F)

        ltc_out, hx = self.ltc(inputs, hx)
        ltc_out = self.tanh(ltc_out)
        d1 = self.dense1(ltc_out)
        d2 = self.dense2(d1)
        out = self.dense3(d2)
        out = out.view(B, S, L)
        out = out[:, -1, :]  

        return out, hx


import time
def train_model(data_folder, input_steps=2, embedding_dim=16, ltc_units=64,
                dense_units=[20,10], lr=1e-3, batch_size=32, epochs=5, neigh_dict=None, test_name=None):
    all_data = load_data_from_txt_folder(data_folder)
    train_data = all_data[:6000]
    dataset = BitrateTimeSeriesDataset(train_data, input_steps, neigh_dict=neigh_dict)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model = TrafficPredictionNetwork(embedding_dim=embedding_dim, ltc_units=ltc_units,
                                     dense_units=dense_units)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    model.train()
    for ep in range(epochs):
        epoch_start = time.perf_counter()
        total_loss = 0.0
        for x_batch, y_batch in loader:
            optimizer.zero_grad()
            preds, _ = model(x_batch)
            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {ep+1}/{epochs}: Loss = {total_loss/len(loader):.6f}, time = {time.perf_counter() - epoch_start:.2f}s")
    save_model(model, f"{test_name}.pth")
    return model, all_data



In [65]:
def check_for_streaks(predictions, real_values, tolerance=0.15):
    streak_info = {}
    num_nodes = len(predictions[0])

    for node in range(num_nodes):
        streak = 0
        for t in range(len(predictions)):

            if abs(predictions[t][node] - real_values[t][node]) <= tolerance * abs(real_values[t][node]):
                streak += 1
                if streak == 5:
                    streak_info[node] = t - 4
                    break
            else:
                streak = 0
    return streak_info, len(streak_info)
    
def plot_predictions(predictions, real_values=None, node=None):
    num_nodes = len(predictions[0])
    node_idx = node if node is not None else random.randrange(num_nodes)

    node_preds = [p[node_idx] for p in predictions]
    avg_preds  = [sum(p)/num_nodes for p in predictions]

    plt.figure(figsize=(14,6))
    plt.subplot(1,2,1)
    plt.plot(node_preds, '-o', label='Predicted')
    if real_values is not None:
        node_reals = [r[node_idx] for r in real_values]
        plt.plot(node_reals, '-x', label='Real')
    plt.title(f'Connection {node_idx} Bitrate')
    plt.xlabel('Timestep')
    plt.ylabel('Bitrate')
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(avg_preds, '-o', label='Avg Predicted')
    if real_values is not None:
        avg_reals = [sum(r)/num_nodes for r in real_values]
        plt.plot(avg_reals, '-x', label='Avg Real')
    plt.title('Average Bitrate Across All Connections')
    plt.xlabel('Timestep')
    plt.ylabel('Average Bitrate')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
def get_all_test_data_for_lnn(root_folder, input_steps=2):
    subfolders = [f for f in os.listdir(root_folder) if f.startswith("processed_7000_")]
    subfolders.sort(key=custom_sort_key_dirs)
    all_data = []
    for subfolder in subfolders:
        data_path = os.path.join(root_folder, subfolder)
        print(f"Loading data from {data_path}")
        data = load_data_from_txt_folder(data_path)
        all_data.append(data)
    return all_data


def evaluate_lnn_on_all_sets(
    model,
    all_test_data, name_of_test,
    criterion, neigh_dict,
    *, tolerance=0.15, input_steps=5,
    base_lr=1e-3, detach=5):
    import copy


    summary_results, all_results, all_streak_timesteps = [], [], []
    initial_model_state = copy.deepcopy(model.state_dict())

    for data_sequence in all_test_data:
        
        optimizer = torch.optim.Adam(model.parameters(), lr=base_lr)
        model.load_state_dict(copy.deepcopy(initial_model_state))
        tol_pct, predictions = evaluate_and_update_lnn_simple_new(
            model, data_sequence, optimizer, criterion,
            neigh_dict=neigh_dict,
            tolerance=tolerance, input_steps=input_steps, detach=detach)
        print("done wtih set")

        real_values = np.stack([data_sequence[t][:,0] for t in range(7000, 7051)])
        pred_arr    = np.asarray(predictions)

        rmse_per_ts = np.sqrt(((pred_arr - real_values)**2).mean(axis=1)).tolist()
        mape_per_ts = (np.abs(pred_arr - real_values) /
                       np.clip(np.abs(real_values), 1e-9, None)).mean(axis=1).tolist()

        streak_info, _ = check_for_streaks(pred_arr.tolist(),
                                           real_values.tolist(), tolerance)
        streak_nodes   = set(streak_info)
        streak_ts      = list(streak_info.values())

        summary_results.append({
            "number_of_streak_nodes": len(streak_nodes),
            "average_streak_timestep": float(np.mean(streak_ts)) if streak_ts else None,
            "percentage_within_tolerance": tol_pct,
            "percentage_nodes_with_streak": 100.0*len(streak_nodes)/pred_arr.shape[1],
            "rmse_per_timestep": rmse_per_ts,
            "mape_per_timestep": mape_per_ts,
        })

        all_results.append({"predictions": pred_arr.tolist(),
                            "real_values": real_values.tolist()})
        all_streak_timesteps.extend(streak_ts)

    with open(f"{name_of_test}_results.json", "w") as fh:
        json.dump(all_results, fh, indent=2)

    return all_results, summary_results

In [ ]:
def analyze_lnn_json_results(json_file_path, tolerance=0.15):
    with open(json_file_path, 'r') as f:
        all_results = json.load(f)

    all_metrics = []
    all_streak_timesteps = []

    for result in all_results:
        predictions = result["predictions"]
        real_values = result["real_values"]

        rmse_per_timestep = []
        mape_per_timestep = []
        for t in range(len(predictions)):
            rmse_timestep = []
            mape_timestep = []
            for node in range(len(predictions[0])):
                real = real_values[t][node]
                pred = predictions[t][node]
                rmse = np.sqrt((real - pred) ** 2)
                mape = np.abs((real - pred) / real) * 100 if real != 0 else 0
                rmse_timestep.append(float(rmse))
                mape_timestep.append(float(mape))
            rmse_per_timestep.append(np.mean(rmse_timestep))
            mape_per_timestep.append(np.mean(mape_timestep))

        streak_info, _ = check_for_streaks(predictions, real_values, tolerance)
        streak_timesteps = list(streak_info.values())

        total_within_tolerance = sum(
            abs(p - r) <= tolerance * abs(r)
            for pred_t, real_t in zip(predictions, real_values)
            for p, r in zip(pred_t, real_t)
        )
        total_preds = len(predictions) * len(predictions[0])
        pct_within_tol = (total_within_tolerance / total_preds) * 100
        pct_nodes_with_streak = (len(streak_info) / len(predictions[0])) * 100
        avg_streak_start = np.mean(streak_timesteps) if streak_timesteps else None

        all_metrics.append({
            "rmse": rmse_per_timestep,
            "mape": mape_per_timestep,
            "pct_within_tol": pct_within_tol,
            "pct_nodes_with_streak": pct_nodes_with_streak,
            "avg_streak_start": avg_streak_start
        })
        all_streak_timesteps.extend(streak_timesteps)

    avg_rmse = np.mean([m["rmse"] for m in all_metrics], axis=0)
    avg_mape = np.mean([m["mape"] for m in all_metrics], axis=0)
    avg_within_tol = np.mean([m["pct_within_tol"] for m in all_metrics])
    avg_nodes_with_streak = np.mean([m["pct_nodes_with_streak"] for m in all_metrics])
    avg_streak_time = np.mean(all_streak_timesteps) if all_streak_timesteps else None

    print(f"Avg % within tolerance: {avg_within_tol:.2f}%")
    print(f"Avg % of nodes with streak: {avg_nodes_with_streak:.2f}%")
    print(f"Avg streak start timestep: {avg_streak_time:.2f}" if avg_streak_time else "→ No streaks found.")

    timesteps = list(range(len(avg_rmse)))
    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    plt.plot(timesteps, avg_rmse, label='Average RMSE', marker='o')
    plt.title('Average RMSE per Timestep')
    plt.xlabel('Timestep')
    plt.ylabel('RMSE')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(timesteps, avg_mape, label='Average MAPE', marker='o')
    plt.title('Average MAPE per Timestep')
    plt.xlabel('Timestep')
    plt.ylabel('MAPE')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
def load_timestep_data(file_path):
    df = pd.read_csv(file_path, sep=r'\s+', header=None, names=["bitrate", "connection_id"])
    return df['bitrate'].tolist()

In [ ]:
def detach_hidden_state(h):
    if h is None:
        return None
    if isinstance(h, torch.Tensor):
        return h.detach()
    if isinstance(h, (list, tuple)):
        return type(h)(detach_hidden_state(x) for x in h)
    return h

In [ ]:
import time
import numpy as np
import torch

def evaluate_and_update_lnn_simple_new(
        model, full_data, optimizer, criterion, *,
        neigh_dict,
        test_start=6000, test_end=7100,
        eval_window=(6999, 7050), incremental_step=20,
        tolerance=0.15, input_steps=5, detach=5):

    model.train()
    h = None
    predictions = []
    total_loss = 0.0
    within_tol = 0
    total_eval = 0

    eval_start, eval_end = eval_window
    connection_ids = np.arange(82).reshape(-1, 1)
    detach_every = detach
    buf_x, buf_y = [], []

    start_time = time.perf_counter()

    for t in range(test_start + input_steps, test_end):
        x_seq = []
        for j in range(input_steps, 0, -1):
            bitrate = full_data[t - j]
            frame = np.concatenate([bitrate, connection_ids], axis=1)
            x_seq.append(frame)
        x_seq = np.stack(x_seq, axis=0)
        x_seq = np.stack([add_neighbour_channels(f, neigh_dict) for f in x_seq], axis=0)

        x_t = torch.from_numpy(x_seq).float().unsqueeze(0)    
        y_true = torch.from_numpy(full_data[t].flatten()).float()

        step_idx = t - (test_start + input_steps)
        in_eval = (eval_start <= t <= eval_end)

        with torch.set_grad_enabled(in_eval):
            pred, h = model(x_t, h)

        if (step_idx % detach_every == 0) and (h is not None):
            h = h.detach()

        if in_eval:
            loss = criterion(pred.view(-1), y_true)
            total_loss += loss.item()
            predictions.append(pred.view(-1).tolist())

            for p, r in zip(pred.view(-1).tolist(), y_true.tolist()):
                if abs(p - r) <= tolerance * abs(r):
                    within_tol += 1
                total_eval += 1
        buf_x.append(x_t.squeeze(0))  
        buf_y.append(y_true)
        
    duration = time.perf_counter() - start_time
    avg_loss = total_loss / max((eval_end - eval_start + 1), 1)
    tol_pct = 100 * within_tol / total_eval if total_eval > 0 else 0.0

    print(f"Eval complete: loss={avg_loss:.6f}, within_tol={tol_pct:.2f}%, time={duration:.2f}s")
    return tol_pct, predictions


In [ ]:
from typing import Dict, Tuple, List

def _enumerate_links(matrix: np.ndarray) -> Tuple[Dict[Tuple[int, int], int], Dict[int, Tuple[int, int]]]:
    id2link = {}
    link2id = {}
    idx = 0
    n = matrix.shape[0]
    for u in range(n):
        for v in range(n):
            if u == v or matrix[u, v] == 0:
                continue
            link2id[(u, v)] = idx
            id2link[idx] = (u, v)
            idx += 1
    return link2id, id2link


def build_neighbor_dict(matrix: np.ndarray, k: int = 7, mode: str = 'out') -> Tuple[Dict[int, List[int]], int, Dict[int, Tuple[int, int]]]:

    assert mode in {'out', 'in', 'both'}, "mode must be 'out', 'in', or 'both'"

    link2id, id2link = _enumerate_links(matrix)
    num_links = len(id2link)
    PAD_ID = 82 

    outlinks = {u: [] for u in range(matrix.shape[0])}
    inlinks = {v: [] for v in range(matrix.shape[0])}
    for (u, v), lid in link2id.items():
        outlinks[u].append(lid)
        inlinks[v].append(lid)

    neigh: Dict[int, List[int]] = {}
    for lid, (u, v) in id2link.items():
        nbrs = []
        if mode in {'out', 'both'}:
            nbrs += [n for n in outlinks[u] if n != lid]
        if mode in {'in', 'both'}:
            nbrs += [n for n in inlinks[v] if n != lid]
        seen = set()
        uniq = [n for n in nbrs if not (n in seen or seen.add(n))]
        padded = (uniq + [PAD_ID] * k)[:k]
        neigh[lid] = padded

    return neigh, PAD_ID, id2link


def add_neighbour_channels(arr: np.ndarray, neigh_dict: Dict[int, List[int]]) -> np.ndarray:
    L = arr.shape[0]
    k = len(next(iter(neigh_dict.values())))
    neighs = np.empty((L, k), dtype=int)
    for i in range(L):
        neighs[i] = neigh_dict[int(arr[i, 1])]
    return np.concatenate([arr, neighs], axis=1)





In [ ]:
def make_static_feat_template(neigh_dict, k=7):
    L = 82
    conn_ids = np.arange(L).reshape(-1, 1)
    neighbors = np.vstack([neigh_dict[lid] for lid in range(L)])

    id_block = np.concatenate([conn_ids, neighbors], axis=1)  
    zero_bitrate = np.zeros((L, 1)) 

    template = np.concatenate([zero_bitrate, id_block], axis=1)
    return template



In [ ]:
from pathlib import Path

def save_model(model: torch.nn.Module, path: str | Path) -> None:
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    torch.save(model.state_dict(), path)

def load_model(model_cls, checkpoint_path: str | Path, *model_args, **model_kwargs):
    model = model_cls(*model_args, **model_kwargs)
    state_dict = torch.load(checkpoint_path, map_location="cpu")
    model.load_state_dict(state_dict)
    model.eval()
    return model


In [ ]:

mat = np.loadtxt("Zbiory danych/matrix.net", dtype=int)
neigh, PAD_ID, id2link = build_neighbor_dict(mat, k=7, mode='out')

L = 82
sample = np.zeros((L, 2))
sample[:, 1] = np.arange(L) 

sample_aug = add_neighbour_channels(sample, neigh)


static_template = make_static_feat_template(neigh, k=7)   



In [ ]:
def show_neigh_table(neigh_dict: Dict[int, List[int]], id2link: Dict[int, Tuple[int, int]], limit: int = 10):
    print("link_id  (u→v)   neighbours")
    print("-" * 40)
    for lid in list(neigh_dict.keys())[:limit]:
        u, v = id2link[lid]
        print(f"{lid:3d}     ({u:2d}→{v:2d})   {neigh_dict[lid]}")



In [ ]:
data_folder = 'Zbiory danych/polaczenie_bitrate_7000/normalized/processed_7000_0'
test_name = "LNN-historia-sąasiedzi"
root_folder = 'Zbiory danych/polaczenie_bitrate_7000/normalized'
history = 3
detach_step = 5

In [ ]:

model, all_data = train_model(data_folder, neigh_dict=neigh, input_steps=history, embedding_dim=16, ltc_units=64,
                              dense_units=[20, 10], lr=1e-3, batch_size=32, epochs=5, test_name=test_name)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [ ]:
tol_pct, preds = evaluate_and_update_lnn_simple_new(
    model=model,
    full_data=all_data,
    optimizer=optimizer,
    criterion=criterion,
    test_start=6000,
    test_end=7050,
    eval_window=(7000, 7050),
    incremental_step=20,
    tolerance=0.15,
    input_steps=2,
    neigh_dict=neigh,
    detach= detach_step
)

print(f"Within tolerance: {tol_pct:.2f}%")

In [ ]:
real_vals = [arr.flatten().tolist() for arr in all_data[7000:7050]]
print(len(real_vals))
streaks, num_streaks = check_for_streaks(preds, real_vals)
print(f"Streaks found in {num_streaks} nodes: {streaks}")

plot_predictions(preds, real_vals)

print(f"Percentage of predictions within tolerance: {tol_pct:.2f}%")

avg_streak_start = (
                sum(streaks.values()) / len(streaks) if streaks else float('inf')
            )
print(f"Average Streak Start: {avg_streak_start:.2f}")

In [ ]:
all_test_data = get_all_test_data_for_lnn(root_folder, input_steps=history)

In [ ]:
trained_path = test_name + ".pth"
model = load_model(
    TrafficPredictionNetwork,
    trained_path,
    embedding_dim = 16,
    ltc_units     = 64,
    dense_units   = [20, 10]
)

In [ ]:

all_results, summary = evaluate_lnn_on_all_sets(
    model=model,
    all_test_data=all_test_data,
    name_of_test=test_name,
    tolerance=0.15,
    neigh_dict=neigh,
    criterion=criterion,
    input_steps=history
)

In [ ]:
overall_within_tolerance = np.mean([r["percentage_within_tolerance"] for r in summary])
overall_streak_percentage = np.mean([r["percentage_nodes_with_streak"] for r in summary])
avg_streak_start = np.mean([r["average_streak_timestep"] for r in summary if r["average_streak_timestep"] is not None])

print(f"Avg % within tolerance: {overall_within_tolerance:.2f}%")
print(f"Avg % of nodes with streak: {overall_streak_percentage:.2f}%")
print(f"Avg streak start timestep: {avg_streak_start:.2f}")
print(f"Saved results to: {test_name}_results.json")

In [ ]:
analyze_lnn_json_results(
    json_file_path= test_name + "_results.json",
    tolerance=0.15
)